In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point


plt.style.use('ggplot')
import pyreadr
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import f1_score, classification_report, roc_auc_score, RocCurveDisplay
from sklearn.neighbors import KNeighborsClassifier


from tqdm import tqdm
import re
import tqdm

In [119]:
traindf = pd.read_csv('dataset/train.csv')
testdf = pd.read_csv('dataset/test.csv')
weatherdf = pd.read_csv('dataset/weather.csv')

In [120]:
traindf['Date'] = pd.to_datetime(traindf['Date'])
traindf.set_index('Date',inplace=True)

weatherdf['Date'] = pd.to_datetime(weatherdf['Date'])
weatherdf.set_index('Date', inplace=True)

In [121]:
#merging train and weather data.
train = traindf.join(weatherdf, how='left')
train.replace('M',np.NaN,inplace=True)
train.replace('-',np.NaN,inplace=True)
train.replace('  T', 1,inplace=True)

In [122]:
train.isna().sum()

Address                       0
Species                       0
Block                         0
Street                        0
Trap                          0
AddressNumberAndStreet        0
Latitude                      0
Longitude                     0
AddressAccuracy               0
NumMosquitos                  0
WnvPresent                    0
Station                       0
Tmax                          0
Tmin                          0
Tavg                          0
Depart                    10506
DewPoint                      0
WetBulb                      93
Heat                          0
Cool                          0
Sunrise                   10506
Sunset                    10506
CodeSum                       0
Depth                     10506
Water1                    21012
SnowFall                  10506
PrecipTotal                  46
StnPressure                  93
SeaLevel                      0
ResultSpeed                   0
ResultDir                     0
AvgSpeed

In [123]:
train.drop(columns=['Depart','Sunrise','Sunset','Depth','Water1','SnowFall'],inplace=True)

In [124]:
train.dtypes

Address                    object
Species                    object
Block                       int64
Street                     object
Trap                       object
AddressNumberAndStreet     object
Latitude                  float64
Longitude                 float64
AddressAccuracy             int64
NumMosquitos                int64
WnvPresent                  int64
Station                     int64
Tmax                        int64
Tmin                        int64
Tavg                       object
DewPoint                    int64
WetBulb                    object
Heat                       object
Cool                       object
CodeSum                    object
PrecipTotal                object
StnPressure                object
SeaLevel                   object
ResultSpeed               float64
ResultDir                   int64
AvgSpeed                   object
dtype: object

In [125]:
train.drop(columns=['Street','Trap','AddressNumberAndStreet','Latitude','Longitude',
                    'Tavg','WetBulb','Cool','CodeSum','PrecipTotal','StnPressure','SeaLevel','ResultSpeed','AvgSpeed'],inplace=True)

In [126]:
#train['PrecipTotal'].fillna(train['PrecipTotal'].mean(),inplace=True)

In [127]:
train.head(3)

,Address,Species,Block,AddressAccuracy,NumMosquitos,WnvPresent,Station,Tmax,Tmin,DewPoint,Heat,ResultDir
Date,,,,,,,,,,,,
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,9,1,0,1,88,60,58,0,18
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,9,1,0,2,88,65,59,0,16
2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,9,1,0,1,88,60,58,0,18


In [128]:
train['Address'][:1]

Date
2007-05-29    4100 North Oak Park Avenue, Chicago, IL 60634,...
Name: Address, dtype: object

In [129]:
range(len(train['Address']))

range(0, 21012)

In [130]:
for i in range(len(train['Address'])):
    match = re.search(r"\b\d{5}(?:[-\s]\d{4})?\b", train['Address'][i])
    if match:
        train['Address'][i] = match.group(0)

/var/folders/yq/ln25zktd7l51hrfl56g5fvl00000gn/T/ipykernel_33570/272782363.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Address'][i] = match.group(0)


In [131]:
train['Address']

Date
2007-05-29    60634
2007-05-29    60634
2007-05-29    60634
2007-05-29    60634
2007-05-29    60646
              ...  
2013-09-26    60622
2013-09-26    60631
2013-09-26    60631
2013-09-26    60621
2013-09-26    60621
Name: Address, Length: 21012, dtype: object

In [132]:
train['Address'].value_counts()

60666                                     1500
60617                                     1468
South Doty Avenue, Chicago, IL, USA       1084
60634                                     1004
60638                                      910
                                          ... 
60615                                       40
60827                                       34
60647                                       34
West 37th Place, Chicago, IL, USA           28
South Wallace Street, Chicago, IL, USA      16
Name: Address, Length: 62, dtype: int64

In [133]:
train.head(3)

,Address,Species,Block,AddressAccuracy,NumMosquitos,WnvPresent,Station,Tmax,Tmin,DewPoint,Heat,ResultDir
Date,,,,,,,,,,,,
2007-05-29,60634,CULEX PIPIENS/RESTUANS,41,9,1,0,1,88,60,58,0,18
2007-05-29,60634,CULEX PIPIENS/RESTUANS,41,9,1,0,2,88,65,59,0,16
2007-05-29,60634,CULEX RESTUANS,41,9,1,0,1,88,60,58,0,18


In [134]:
#Dummify
expected_dummy_features = ['Species','Address'] #dummify features
train = pd.get_dummies(train,columns=expected_dummy_features, drop_first=True)

In [135]:
train['Block'].unique()

array([41, 62, 79, 15, 25, 11, 21, 22, 17, 53, 65, 75, 89, 91, 36, 72, 38,
       49, 58, 50, 14, 10, 42, 40, 43, 28, 18, 70, 35, 93, 37, 45, 46, 60,
       61, 27, 68, 30, 96, 64, 12, 20, 33, 52, 67, 73, 39, 47, 55, 71, 77,
       63, 13, 81, 24, 80, 82, 98, 66, 48, 51, 90, 29, 34])

In [136]:
train['AddressAccuracy'].unique()

array([9, 8, 5, 3])

In [137]:
train['NumMosquitos'].unique()

array([ 1,  4,  2,  3,  5,  9,  7, 10,  8,  6, 19, 20, 25, 16, 11, 12, 28,
       18, 50, 35, 14, 22, 21, 37, 27, 13, 39, 29, 15, 17, 34, 26, 32, 47,
       44, 23, 46, 48, 42, 33, 45, 24, 41, 38, 40, 36, 43, 49, 30, 31])

In [138]:
train['WnvPresent'].unique()

array([0, 1])

In [139]:
train['Station'].unique()

array([1, 2])

In [140]:
train['Tmax'].unique()

array([88, 64, 63, 92, 91, 70, 71, 79, 77, 85, 83, 82, 90, 87, 84, 80, 81,
       89, 67, 66, 76, 62, 60, 61, 74, 72, 86, 78, 75, 59, 69, 68, 93, 94,
       58, 57, 96, 97])

In [141]:
train['Tmin'].unique()

array([60, 65, 47, 51, 70, 73, 56, 53, 58, 62, 63, 69, 71, 72, 74, 67, 66,
       46, 68, 49, 50, 54, 55, 48, 43, 44, 57, 61, 64, 59, 41, 76, 79, 45,
       52, 75])

In [142]:
train['DewPoint'].unique()

array([58, 59, 48, 47, 69, 55, 56, 50, 51, 68, 63, 62, 61, 60, 67, 66, 73,
       71, 72, 64, 65, 54, 70, 42, 43, 57, 44, 46, 39, 40, 49, 52, 38, 41,
       45, 53])

In [143]:
train['Heat'].unique()

array(['0', '9', '8', '2', '1', '7', '5', '11', '13', '3', '4', '15',
       '10', '14', '12'], dtype=object)

In [144]:
train['Heat'] = train['Heat'].astype(int)

In [145]:
train['Heat'].unique()

array([ 0,  9,  8,  2,  1,  7,  5, 11, 13,  3,  4, 15, 10, 14, 12])

In [146]:
train['ResultDir'].unique()

array([18, 16,  5,  3, 22, 21,  2, 14, 12, 30, 24,  1, 36, 25, 23,  9, 13,
       26,  6,  4, 29, 27,  7, 32, 31, 17, 20, 19, 28, 11, 15,  8, 33, 10,
       35, 34])

In [147]:
train.dtypes

Block                                                int64
AddressAccuracy                                      int64
NumMosquitos                                         int64
WnvPresent                                           int64
Station                                              int64
                                                     ...  
Address_South Torrence Avenue, Chicago, IL, USA      uint8
Address_South Vincennes Avenue, Chicago, IL, USA     uint8
Address_South Wallace Street, Chicago, IL, USA       uint8
Address_West 37th Place, Chicago, IL, USA            uint8
Address_West Garfield Boulevard, Chicago, IL, USA    uint8
Length: 77, dtype: object

In [148]:
Features = [col for col in train._get_numeric_data().columns if col not in ['WnvPresent','NumMosquitos']]

In [149]:
Features

['Block',
 'AddressAccuracy',
 'Station',
 'Tmax',
 'Tmin',
 'DewPoint',
 'Heat',
 'ResultDir',
 'Species_CULEX PIPIENS',
 'Species_CULEX PIPIENS/RESTUANS',
 'Species_CULEX RESTUANS',
 'Species_CULEX SALINARIUS',
 'Species_CULEX TARSALIS',
 'Species_CULEX TERRITANS',
 'Address_60106',
 'Address_60605',
 'Address_60607',
 'Address_60608',
 'Address_60609',
 'Address_60611',
 'Address_60612',
 'Address_60613',
 'Address_60614',
 'Address_60615',
 'Address_60617',
 'Address_60618',
 'Address_60619',
 'Address_60620',
 'Address_60621',
 'Address_60622',
 'Address_60623',
 'Address_60625',
 'Address_60626',
 'Address_60628',
 'Address_60629',
 'Address_60630',
 'Address_60631',
 'Address_60632',
 'Address_60633',
 'Address_60634',
 'Address_60636',
 'Address_60637',
 'Address_60638',
 'Address_60639',
 'Address_60640',
 'Address_60641',
 'Address_60642',
 'Address_60643',
 'Address_60645',
 'Address_60646',
 'Address_60647',
 'Address_60649',
 'Address_60651',
 'Address_60652',
 'Address_60

In [150]:
X = train[Features] #Define X as text
y = train['WnvPresent']

In [151]:
#Split train and validation data to 67% and 33%
X_train, X_valid, y_train, y_valid = train_test_split(X,y,
                                                    test_size=0.33,
                                                    stratify=y,)

In [152]:
lr = LogisticRegression(max_iter=3000)

In [153]:
lr.fit(X_train,y_train)

LogisticRegression(max_iter=3000)

In [154]:
lr.score(X_train,y_train)

0.9475777809347918

In [155]:
lr.score(X_valid,y_valid)

0.9475050475915777

In [156]:
lr.predict_proba(X_valid)

array([[0.87117797, 0.12882203],
       [0.89899728, 0.10100272],
       [0.95622188, 0.04377812],
       ...,
       [0.92736281, 0.07263719],
       [0.93706176, 0.06293824],
       [0.95723104, 0.04276896]])

In [157]:
testdf.head(3)

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [158]:
testdf['Date'] = pd.to_datetime(testdf['Date'])
testdf.set_index('Date', inplace=True)

In [159]:
testdf.head(3)

,Id,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
Date,,,,,,,,,,
2008-06-11,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2008-06-11,2,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2008-06-11,3,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [160]:
weather1 = weatherdf[weatherdf.Station ==1].copy()

In [161]:
#merging train and weather data.
test = testdf.join(weather1, how='left')
test.replace('M',np.NaN,inplace=True)
test.replace('-',np.NaN,inplace=True)
test.replace('  T', 1,inplace=True)

In [162]:
test.head(3)

,Id,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
Date,,,,,,,,,,,,,,,,,,,,,
2008-06-11,1,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,,0,NaN,0.0,0.00,29.28,29.99,8.9,18,10.0
2008-06-11,2,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,,0,NaN,0.0,0.00,29.28,29.99,8.9,18,10.0
2008-06-11,3,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,,0,NaN,0.0,0.00,29.28,29.99,8.9,18,10.0


In [163]:
for i in range(len(test['Address'])):
    match = re.search(r"\b\d{5}(?:[-\s]\d{4})?\b", test['Address'][i])
    if match:
        test['Address'][i] = match.group(0)

/var/folders/yq/ln25zktd7l51hrfl56g5fvl00000gn/T/ipykernel_33570/3947352977.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Address'][i] = match.group(0)


In [164]:
test.drop(columns=['Depart','Sunrise','Sunset','Depth','Water1','SnowFall'],inplace=True)

In [165]:
test.drop(columns=['Street','Trap','AddressNumberAndStreet','Latitude','Longitude',
                    'Tavg','WetBulb','Cool','CodeSum','PrecipTotal','StnPressure','SeaLevel','ResultSpeed','AvgSpeed'],inplace=True)

In [166]:
test.head(3)

,Id,Address,Species,Block,AddressAccuracy,Station,Tmax,Tmin,DewPoint,Heat,ResultDir
Date,,,,,,,,,,,
2008-06-11,1,60634,CULEX PIPIENS/RESTUANS,41,9,1,86,61,56,0,18
2008-06-11,2,60634,CULEX RESTUANS,41,9,1,86,61,56,0,18
2008-06-11,3,60634,CULEX PIPIENS,41,9,1,86,61,56,0,18


In [167]:
#Dummify
expected_dummy_features = ['Species','Address'] #dummify features
test = pd.get_dummies(test,columns=expected_dummy_features, drop_first=True)

In [168]:
Features = [col for col in train._get_numeric_data().columns if col not in ['WnvPresent','NumMosquitos']]

In [169]:
X_test = test[Features] #Define X as text

In [170]:
lr.predict_proba(X_test)

array([[0.93644669, 0.06355331],
       [0.9815252 , 0.0184748 ],
       [0.91141946, 0.08858054],
       ...,
       [0.97024029, 0.02975971],
       [0.96704643, 0.03295357],
       [0.96704643, 0.03295357]])

In [171]:
df_submit = pd.DataFrame(test, columns=['Id']) #create data frame with ids
df_submit['WnvPresent'] = lr.predict_proba(X_test)[:,1] #add predicted price
df_submit.head(2) #check

,Id,WnvPresent
Date,,
2008-06-11,1,0.063553
2008-06-11,2,0.018475


In [172]:
df_submit.to_csv('submission_model2.csv', index=False) #export